<a href="https://colab.research.google.com/github/amitadhainje/LearningBERT/blob/master/NLP_Interpretability_2020_06_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
files.upload()

Saving IMDB Dataset.csv to IMDB Dataset.csv


In [0]:
from __future__ import print_function

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, Bidirectional, LSTM
import numpy as np
import pandas as pd

df = pd.read_csv('IMDB Dataset.csv', nrows=10000)
df.sentiment = df.sentiment.map({ 'positive' : 1, 'negative' : 0})
#df.loc[df['sentiment'] == 'positive', sentiment] = 1
from sklearn.model_selection import train_test_split

texts_train, texts_test, y_train, y_test = train_test_split(df["review"].values, df['sentiment'].values, random_state=42)

In [0]:
max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

tokenizer = Tokenizer(num_words=max_features)
trainingData = tokenizer.fit_on_texts(texts_train)

train_data = tokenizer.texts_to_sequences(texts_train)
test_data = tokenizer.texts_to_sequences(texts_test)

padded_train_data = pad_sequences(train_data, maxlen=80, padding="post")
padded_test_data = pad_sequences(test_data, maxlen=80, padding="post")


In [72]:
batch_size = 64
max_features = len(tokenizer.word_index) + 1

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(max_features, 128))
model.add(tf.keras.layers.LSTM(128, dropout=0.5, recurrent_dropout=0.5))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
print (model.summary())
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(padded_train_data, y_train, epochs=2, batch_size=batch_size)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         6727552   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 6,859,265
Trainable params: 6,859,265
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/2
118/118 [==============================] - 43s 362ms/step - loss: 0.5909 - accuracy: 0.6683
Epoch 2/2
118/118 [==============================] - 42s 358ms/step - loss: 0.3083 - accuracy: 0.8735


In [73]:
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(padded_test_data, y_test, batch_size=64)
print('test loss, test acc:', results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print('\n# Generate predictions for 3 samples')
predictions = model.predict(padded_test_data[:3])
print('predictions shape:', predictions.shape)


# Evaluate on test data
40/40 [==============================] - 2s 41ms/step - loss: 0.4357 - accuracy: 0.7916
test loss, test acc: [0.43573594093322754, 0.7915999889373779]

# Generate predictions for 3 samples
predictions shape: (3, 1)


In [74]:
!mkdir /content/saved_model
model.save("/content/saved_model/")

mkdir: cannot create directory ‘/content/saved_model’: File exists
INFO:tensorflow:Assets written to: /content/saved_model/assets


INFO:tensorflow:Assets written to: /content/saved_model/assets


In [9]:
!ls -ltr /content/saved_model

total 588
drwxr-xr-x 2 root root   4096 Jun  7 12:46 variables
drwxr-xr-x 2 root root   4096 Jun  7 12:46 assets
-rw-r--r-- 1 root root 592535 Jun  7 12:46 saved_model.pb


In [0]:
def serialize_example(reviews):
  """
  Creates a tf.Example message ready to be written to a file.
  """
  # Create a dictionary mapping the feature name to the tf.Example-compatible
  # data type.
  feature = {
      'reviews': _bytes_feature(reviews),
  }

  # Create a Features message using tf.train.Example.

  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

In [12]:
train_tensor_records = tf.data.Dataset.from_tensor_slices((padded_train_data))
serialized_features_dataset = train_tensor_records.map(tf.io.serialize_tensor)
for c in train_tensor_records.take(1):
    print (c)
    print (tf.io.serialize_tensor(c))

tf.Tensor(
[ 1238  4145   766     1    51    21   101    43    11     1   397     9
   201    42    22    57  6155    14     3    74   172   522    38     3
   523    15     9   199    52    67  1226   140     1   365     4     1
 15949     2   136  1072   127     9   438  1958    21    20     1  6387
   245     4   124   122    44     9    66    32 11543  1444    10   372
    11    64     5    84    37   178     5    83    74    11     6     2
    89   129    10   165  3092  3507   291   154], shape=(80,), dtype=int32)
tf.Tensor(b'\x08\x03\x12\x04\x12\x02\x08P"\xc0\x02\xd6\x04\x00\x001\x10\x00\x00\xfe\x02\x00\x00\x01\x00\x00\x003\x00\x00\x00\x15\x00\x00\x00e\x00\x00\x00+\x00\x00\x00\x0b\x00\x00\x00\x01\x00\x00\x00\x8d\x01\x00\x00\t\x00\x00\x00\xc9\x00\x00\x00*\x00\x00\x00\x16\x00\x00\x009\x00\x00\x00\x0b\x18\x00\x00\x0e\x00\x00\x00\x03\x00\x00\x00J\x00\x00\x00\xac\x00\x00\x00\n\x02\x00\x00&\x00\x00\x00\x03\x00\x00\x00\x0b\x02\x00\x00\x0f\x00\x00\x00\t\x00\x00\x00\xc7\x00\x00\x004\x00\

In [0]:
filename = 'test.tfrecord'
writer = tf.data.experimental.TFRecordWriter(filename)
writer.write(serialized_features_dataset)

In [0]:
def run_and_render(eval_model=None, slice_list=None, slice_idx=0):
  """Runs the model analysis and renders the slicing metrics

  Args:
      eval_model: An instance of tf.saved_model saved with evaluation data
      slice_list: A list of tfma.slicer.SingleSliceSpec giving the slices
      slice_idx: An integer index into slice_list specifying the slice to use

  Returns:
      A SlicingMetricsViewer object if in Jupyter notebook; None if in Colab.
  """
  eval_result = tfma.run_model_analysis(eval_shared_model=eval_model,
                                          data_location="test.tfrecord",
                                          file_format='tfrecords',
                                          slice_spec=slice_list,
                                          output_path='sample_data',
                                          extractors=None)
  return tfma.view.render_slicing_metrics(eval_result, slicing_spec=slice_list[slice_idx])

In [15]:
!pip install -q tensorflow-model-analysis==0.21.3

import tensorflow as tf
import tensorflow_model_analysis as tfma
print('TFMA version: {}'.format(tfma.version.VERSION_STRING))


     |████████████████████████████████| 1.5MB 2.8MB/s 
     |████████████████████████████████| 112kB 14.3MB/s 
     |████████████████████████████████| 2.0MB 14.8MB/s 
     |████████████████████████████████| 3.5MB 37.0MB/s 
     |████████████████████████████████| 63.2MB 121kB/s 
     |████████████████████████████████| 225kB 35.1MB/s 
     |████████████████████████████████| 81kB 7.8MB/s 
     |████████████████████████████████| 61kB 6.2MB/s 
     |████████████████████████████████| 1.2MB 30.9MB/s 
     |████████████████████████████████| 51kB 4.4MB/s 
     |████████████████████████████████| 440kB 37.3MB/s 
     |████████████████████████████████| 184kB 31.0MB/s 
     |████████████████████████████████| 153kB 28.6MB/s 
     |████████████████████████████████| 92kB 8.1MB/s 
     |████████████████████████████████| 235kB 43.6MB/s 
     |████████████████████████████████| 174kB 43.2MB/s 
     |████████████████████████████████| 215kB 48.7MB/s 
     |████████████████████████████████| 92kB 10.7MB/s 
  

Error importing tfx_bsl_extension.coders. Some tfx_bsl functionalities are not available

In [19]:
import os
eval_model_dir_0 = os.path.join("/content/","saved_model")
eval_shared_model_0 = tfma.default_eval_shared_model(eval_saved_model_path=eval_model_dir_0)
slices = [tfma.slicer.SingleSliceSpec(columns=['review'])]

run_and_render(eval_model=eval_shared_model_0, slice_list=slices, slice_idx=0)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.


RuntimeError: ignored

In [29]:
!saved_model_cli show --dir  "/content/saved_model_2/"

The given SavedModel contains the following tag-sets:
serve


In [30]:
!saved_model_cli show --dir "/content/saved_model_2/" --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['embedding_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, -1)
      name: serving_default_embedding_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [22]:
loaded = tf.saved_model.load("/content/saved_model/")
print(list(loaded.signatures.keys())) 

['serving_default']


In [23]:
infer = loaded.signatures["serving_default"]
print(infer.structured_outputs)

{'dense': TensorSpec(shape=(None, 1), dtype=tf.float32, name='dense')}


In [24]:
import tensorflow as tf
print('TF version: {}'.format(tf.__version__))

TF version: 2.2.0


In [25]:
import tensorflow as tf
import tensorflow_model_analysis as tfma
print('TFMA version: {}'.format(tfma.version.VERSION_STRING))

TFMA version: 0.21.3


In [27]:
#!mkdir /content/saved_model_2
tf.saved_model.save(model, "/content/saved_model_2/")

INFO:tensorflow:Assets written to: /content/saved_model_2/assets


INFO:tensorflow:Assets written to: /content/saved_model_2/assets


In [28]:
!ls -ltr /content/saved_model_2/

total 588
drwxr-xr-x 2 root root   4096 Jun  7 14:52 variables
drwxr-xr-x 2 root root   4096 Jun  7 14:52 assets
-rw-r--r-- 1 root root 592861 Jun  7 14:52 saved_model.pb


In [32]:
import tempfile
model_dir = tempfile.mkdtemp()
print (model_dir)
keras_estimator = tf.keras.estimator.model_to_estimator(keras_model=model, model_dir=model_dir)

/tmp/tmp7v0rgxco
INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using the Keras model provided.


INFO:tensorflow:Using the Keras model provided.


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp7v0rgxco', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp7v0rgxco', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
def parser(x, length, y):
    features = {"x": x}
    return features, y

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((x_train, 80, y_train))
    dataset = dataset.shuffle(buffer_size=80)
    dataset = dataset.batch(32)
    dataset = dataset.map(parser)
    dataset = dataset.repeat()
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

In [63]:
def parser(x, y):
    features = {"embedding_input": x}
    return features, y

def input_fn():
    train_tensor_records = tf.data.Dataset.from_tensor_slices((padded_train_data, y_train))
    #train_tensor_records = train_tensor_records.map(parser)
    train_tensor_records = train_tensor_records.batch(32).repeat()
    return train_tensor_records

for features_batch, labels_batch in input_fn().take(1):
  print(features_batch)
  print(labels_batch)


tf.Tensor(
[[ 1238  4145   766 ...  3507   291   154]
 [  386   345     8 ...     1   507   627]
 [   97    10    25 ...    20    11    15]
 ...
 [ 1962   240     3 ...   564   819   154]
 [    1   338  9866 ...    26 18653    96]
 [   79    15  8598 ...     0     0     0]], shape=(32, 80), dtype=int32)
tf.Tensor([0 1 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 1 0 1 1 1], shape=(32,), dtype=int64)


In [61]:
keras_estimator.train(input_fn=input_fn, steps=10)
eval_result = keras_estimator.evaluate(input_fn=input_fn, steps=10)
print('Eval result: {}'.format(eval_result))

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:Saving dict for global step 20: accuracy = 0.99375, global_step = 20, loss = 0.023519631


INFO:tensorflow:Saving dict for global step 20: accuracy = 0.99375, global_step = 20, loss = 0.023519631


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20: /tmp/tmp7v0rgxco/model.ckpt-20


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20: /tmp/tmp7v0rgxco/model.ckpt-20


Eval result: {'accuracy': 0.99375, 'loss': 0.023519631, 'global_step': 20}


In [49]:
!ls -ltr /tmp/tmp7v0rgxco/
#keras_estimator.export_saved_model("",experimental_mode="eval")

total 188272
drwxr-xr-x 2 root root     4096 Jun  7 15:00 keras
-rw-r--r-- 1 root root 10505879 Jun  7 15:21 graph.pbtxt
-rw-r--r-- 1 root root     3262 Jun  7 15:21 model.ckpt-0.data-00001-of-00002
-rw-r--r-- 1 root root 82311204 Jun  7 15:21 model.ckpt-0.data-00000-of-00002
-rw-r--r-- 1 root root     1697 Jun  7 15:21 model.ckpt-0.index
-rw-r--r-- 1 root root  2950477 Jun  7 15:21 model.ckpt-0.meta
-rw-r--r-- 1 root root     3262 Jun  7 15:21 model.ckpt-10.data-00001-of-00002
-rw-r--r-- 1 root root 82311204 Jun  7 15:21 model.ckpt-10.data-00000-of-00002
-rw-r--r-- 1 root root     1697 Jun  7 15:21 model.ckpt-10.index
-rw-r--r-- 1 root root      177 Jun  7 15:21 checkpoint
-rw-r--r-- 1 root root  2950477 Jun  7 15:21 model.ckpt-10.meta
-rw-r--r-- 1 root root 11721461 Jun  7 15:21 events.out.tfevents.1591543301.7f39b6c6ba3f
drwxr-xr-x 2 root root     4096 Jun  7 15:21 eval


In [62]:
import os
from pathlib import Path
str_path = "/content/saved_model_2/"

keras_estimator.export_saved_model(str_path,input_fn)

AttributeError: ignored

In [55]:
import os
#from pathlib import Path
#str_path = "/tmp/tmp7v0rgxco/"
#path = Path(str_path)
!saved_model_cli show --dir  "/tmp/tmp7v0rgxco/"

Traceback (most recent call last):
  File "/usr/local/bin/saved_model_cli", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/tensorflow_core/python/tools/saved_model_cli.py", line 990, in main
    args.func(args)
  File "/usr/local/lib/python2.7/dist-packages/tensorflow_core/python/tools/saved_model_cli.py", line 697, in show
    _show_tag_sets(args.dir)
  File "/usr/local/lib/python2.7/dist-packages/tensorflow_core/python/tools/saved_model_cli.py", line 62, in _show_tag_sets
    tag_sets = saved_model_utils.get_saved_model_tag_sets(saved_model_dir)
  File "/usr/local/lib/python2.7/dist-packages/tensorflow_core/python/tools/saved_model_utils.py", line 88, in get_saved_model_tag_sets
    saved_model = read_saved_model(saved_model_dir)
  File "/usr/local/lib/python2.7/dist-packages/tensorflow_core/python/tools/saved_model_utils.py", line 55, in read_saved_model
    raise IOError("SavedModel file does not exist at: %s" % saved_model_dir)
IOError: Sav

In [76]:
from google.protobuf import text_format

eval_config = text_format.Parse("""
  ## Model information
  model_specs {
    # This assumes a serving model with a "serving_default" signature.
    label_key: "label"
    example_weight_key: "weight"
  }
  ## Post export metric information
  metrics_specs {
    # This adds AUC and as a post training metric. If the model has built in
    # training metrics which also contains AUC, this metric will replace it.
    metrics { class_name: "AUC" }
    # ... other post training metrics ...

    # Plots are also configured here...
    metrics { class_name: "ConfusionMatrixPlot" }
  }
  ## Slicing information
  slicing_specs {}  # overall slice
  slicing_specs {
    feature_keys: ["age"]
  }
""", tfma.EvalConfig())

eval_shared_model = tfma.default_eval_shared_model(
    eval_saved_model_path='/content/saved_model', eval_config=eval_config)

eval_result = tfma.run_model_analysis(
    eval_shared_model=eval_shared_model,
    eval_config=eval_config,
    # This assumes your data is a TFRecords file containing records in the
    # tf.train.Example format.
    data_location='/content/test.tfrecord',
    output_path='/content/output')

tfma.view.render_slicing_metrics(eval_result)

AttributeError: ignored

In [65]:
!ls -ltr /content
!mkdir /content/output

total 67224
drwxr-xr-x 1 root root     4096 May 29 18:19  sample_data
-rw-r--r-- 1 root root 66212309 Jun  7 12:43 'IMDB Dataset.csv'
drwxr-xr-x 4 root root     4096 Jun  7 12:46  saved_model
-rw-r--r-- 1 root root  2602500 Jun  7 12:49  test.tfrecord
drwxr-xr-x 6 root root     4096 Jun  7 15:47  saved_model_2
